# Notebook 31: Asynchronous Pipeline## Phase 6: Streams & Concurrency**Learning Objectives:**- Understand overlapping- Learn pipeline- Master compute and transfer- Apply concepts in practical scenarios- Measure and analyze performance

## Concept: Asynchronous Pipeline**Topics Covered:**- overlapping- pipeline- compute and transfer**Key Concepts:**This notebook covers overlapping in the context of Phase 6: Streams & Concurrency.

## Example 1: Basic Asynchronous Pipeline

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)


__global__ void processKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        for (int i = 0; i < 50; i++)
            data[idx] = sqrtf(fabsf(data[idx]) + 1.0f);
    }
}

int main() {
    printf("=== Async Pipeline ===\n\n");
    const int N = 1 << 20;
    const int nChunks = 8;
    size_t bytes = N * sizeof(float);
    size_t chunkBytes = bytes / nChunks;

    float *h_data = (float*)malloc(bytes);
    for (int i = 0; i < N; i++) h_data[i] = (float)i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, bytes));

    cudaStream_t stream;
    CUDA_CHECK(cudaStreamCreate(&stream));

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));
    CUDA_CHECK(cudaEventRecord(start));

    int chunkSize = N / nChunks;
    for (int i = 0; i < nChunks; i++) {
        int offset = i * chunkSize;
        CUDA_CHECK(cudaMemcpyAsync(d_data + offset, h_data + offset,
                   chunkBytes, cudaMemcpyHostToDevice, stream));
        processKernel<<<(chunkSize + 255) / 256, 256, 0, stream>>>(d_data + offset, chunkSize);
        CUDA_CHECK(cudaMemcpyAsync(h_data + offset, d_data + offset,
                   chunkBytes, cudaMemcpyDeviceToHost, stream));
    }

    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaStreamSynchronize(stream));
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    printf("Pipelined processing with %d chunks\n", nChunks);
    printf("Time: %.3f ms\n", ms);

    cudaStreamDestroy(stream);
    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start); cudaEventDestroy(stop);
    return 0;
}


## Practical ExerciseComplete the following exercises to practice the concepts learned.

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", \
                    __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

__global__ void kernel(float *data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

int main() {
    printf("=== Async Pipeline ===\n\n");

    int n = 1000000;
    size_t size = n * sizeof(float);

    float *h_data = (float*)malloc(size);
    for (int i = 0; i < n; i++) h_data[i] = i;

    float *d_data;
    CUDA_CHECK(cudaMalloc(&d_data, size));
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int threads = 256;
    int blocks = (n + threads - 1) / threads;

    cudaEventRecord(start);
    kernel<<<blocks, threads>>>(d_data, n);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    CUDA_CHECK(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));

    printf("Processed %d elements in %.2f ms\n", n, ms);
    printf("Bandwidth: %.2f GB/s\n", (size * 2 / 1e9) / (ms / 1000.0));

    free(h_data);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

## Key Takeaways

1. Overlap transfer, compute, and transfer back
2. Pipeline multiple batches
3. Maximize GPU utilization
4. Requires pinned memory and streams

## Next StepsContinue to: **32_next_topic.ipynb**

## Notes*Use this space to write your own notes and observations:*------